In [ ]:
import optuna
from functools import partial

In [ ]:
def objective_lgbr(trial, X, y):
    param = {
        'n_estimators': 2000,
        'max_depth': trial.suggest_int('max_depth', 3, 11),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.005, 0.01),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 100),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 100),
        'n_jobs': -1
    }
    train_scores, test_scores = [], []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    model = XGBRegressor(**param)
    for train_idx, test_idx in kf.split(X):
        tmp_X_train, tmp_X_test = X_train.iloc[train_idx, :], X_train.iloc[test_idx, :]
        tmp_y_train, tmp_y_test = y_train[train_idx], y_train[test_idx]
        model.fit(tmp_X_train, tmp_y_train,
                 eval_set=[(tmp_X_test, tmp_y_test)], eval_metric=['rmse'],
                 early_stopping_rounds=30, verbose=0,
                 callbacks=[optuna.integration.XGBoostPruningCallback(trial, 'rmse')])
        train_score = np.sqrt(mse(tmp_y_train, model.predict(tmp_X_train)))
        test_score = np.sqrt(mse(tmp_y_test, model.predict(tmp_X_test)))
        train_scores.append(train_score)
        test_scores.append(test_score)
    train_score = np.array(train_scores).mean()
    test_score = np.array(test_scores).mean()
    print(f'train score: {train_score}')
    print(f'test score: {test_score}')
    return test_score

In [ ]:
optimizer_lgbr = partial(objective_lgbr, X=X_train, y=y_train)
study_lgbr = optuna.create_study(direction='minimize')
study_lgbr.optimize(optimizer_lgbr, n_trials=100)

In [ ]:
optuna.visualization.plot_optimization_history(study_lgbr)

In [ ]:
optuna.visualization.plot_slice(study_lgbr)